# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 20 2022 12:58PM,242342,10,SONSB0001876,"Nextsource Biotechnology, LLC",Released
1,Jul 20 2022 12:57PM,242340,21,621415,"NBTY Global, Inc.",Released
2,Jul 20 2022 12:57PM,242339,10,8295390,Eye Pharma Inc,Released
3,Jul 20 2022 12:51PM,242338,19,60017832,"GUSA Granules USA, Inc.",Released
4,Jul 20 2022 12:31PM,242336,12,HH7202022,Hush Hush,Released
5,Jul 20 2022 12:30PM,242335,10,8299700,MedStone Pharma LLC,Released
6,Jul 20 2022 12:30PM,242335,10,8299701,MedStone Pharma LLC,Released
7,Jul 20 2022 12:30PM,242335,10,8299702,MedStone Pharma LLC,Released
8,Jul 20 2022 12:30PM,242335,10,8299703,MedStone Pharma LLC,Released
9,Jul 20 2022 12:30PM,242335,10,8299704,MedStone Pharma LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,242336,Released,1
34,242338,Released,1
35,242339,Released,1
36,242340,Released,1
37,242342,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
242336,NaN,1.0
242338,NaN,1.0
242339,NaN,1.0
242340,NaN,1.0
242342,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
242224,1.0,0.0
242231,1.0,0.0
242233,0.0,19.0
242252,0.0,1.0
242263,0.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
242224,1,0
242231,1,0
242233,0,19
242252,0,1
242263,0,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,242224,1,0
1,242231,1,0
2,242233,0,19
3,242252,0,1
4,242263,0,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,242224,1,
1,242231,1,
2,242233,,19
3,242252,,1
4,242263,,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 20 2022 12:58PM,242342,10,"Nextsource Biotechnology, LLC"
1,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc."
2,Jul 20 2022 12:57PM,242339,10,Eye Pharma Inc
3,Jul 20 2022 12:51PM,242338,19,"GUSA Granules USA, Inc."
4,Jul 20 2022 12:31PM,242336,12,Hush Hush
5,Jul 20 2022 12:30PM,242335,10,MedStone Pharma LLC
16,Jul 20 2022 12:30PM,242334,20,Alumier Labs Inc.
25,Jul 20 2022 12:28PM,242333,10,Emerginnova
31,Jul 20 2022 12:25PM,242329,22,"NBTY Global, Inc."
32,Jul 20 2022 12:09PM,242328,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 20 2022 12:58PM,242342,10,"Nextsource Biotechnology, LLC",,1
1,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc.",,1
2,Jul 20 2022 12:57PM,242339,10,Eye Pharma Inc,,1
3,Jul 20 2022 12:51PM,242338,19,"GUSA Granules USA, Inc.",,1
4,Jul 20 2022 12:31PM,242336,12,Hush Hush,,1
5,Jul 20 2022 12:30PM,242335,10,MedStone Pharma LLC,,11
6,Jul 20 2022 12:30PM,242334,20,Alumier Labs Inc.,,9
7,Jul 20 2022 12:28PM,242333,10,Emerginnova,,6
8,Jul 20 2022 12:25PM,242329,22,"NBTY Global, Inc.",,1
9,Jul 20 2022 12:09PM,242328,19,"GCH Granules USA, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 20 2022 12:58PM,242342,10,"Nextsource Biotechnology, LLC",1,
1,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc.",1,
2,Jul 20 2022 12:57PM,242339,10,Eye Pharma Inc,1,
3,Jul 20 2022 12:51PM,242338,19,"GUSA Granules USA, Inc.",1,
4,Jul 20 2022 12:31PM,242336,12,Hush Hush,1,
5,Jul 20 2022 12:30PM,242335,10,MedStone Pharma LLC,11,
6,Jul 20 2022 12:30PM,242334,20,Alumier Labs Inc.,9,
7,Jul 20 2022 12:28PM,242333,10,Emerginnova,6,
8,Jul 20 2022 12:25PM,242329,22,"NBTY Global, Inc.",1,
9,Jul 20 2022 12:09PM,242328,19,"GCH Granules USA, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 20 2022 12:58PM,242342,10,"Nextsource Biotechnology, LLC",1,
1,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc.",1,
2,Jul 20 2022 12:57PM,242339,10,Eye Pharma Inc,1,
3,Jul 20 2022 12:51PM,242338,19,"GUSA Granules USA, Inc.",1,
4,Jul 20 2022 12:31PM,242336,12,Hush Hush,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 20 2022 12:58PM,242342,10,"Nextsource Biotechnology, LLC",1.0,NaN
1,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc.",1.0,NaN
2,Jul 20 2022 12:57PM,242339,10,Eye Pharma Inc,1.0,NaN
3,Jul 20 2022 12:51PM,242338,19,"GUSA Granules USA, Inc.",1.0,NaN
4,Jul 20 2022 12:31PM,242336,12,Hush Hush,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 20 2022 12:58PM,242342,10,"Nextsource Biotechnology, LLC",1.0,0.0
1,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc.",1.0,0.0
2,Jul 20 2022 12:57PM,242339,10,Eye Pharma Inc,1.0,0.0
3,Jul 20 2022 12:51PM,242338,19,"GUSA Granules USA, Inc.",1.0,0.0
4,Jul 20 2022 12:31PM,242336,12,Hush Hush,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2665453,65.0,4.0
12,242336,1.0,0.0
15,242311,4.0,2.0
16,969164,4.0,0.0
18,242224,0.0,1.0
19,2180689,11.0,1.0
20,726869,31.0,10.0
21,969299,4.0,0.0
22,242329,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2665453,65.0,4.0
1,12,242336,1.0,0.0
2,15,242311,4.0,2.0
3,16,969164,4.0,0.0
4,18,242224,0.0,1.0
5,19,2180689,11.0,1.0
6,20,726869,31.0,10.0
7,21,969299,4.0,0.0
8,22,242329,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,65.0,4.0
1,12,1.0,0.0
2,15,4.0,2.0
3,16,4.0,0.0
4,18,0.0,1.0
5,19,11.0,1.0
6,20,31.0,10.0
7,21,4.0,0.0
8,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,65.0
1,12,Released,1.0
2,15,Released,4.0
3,16,Released,4.0
4,18,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Executing,4.0,0.0,2.0,0.0,1.0,1.0,10.0,0.0,0.0
Released,65.0,1.0,4.0,4.0,0.0,11.0,31.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Executing,4.0,0.0,2.0,0.0,1.0,1.0,10.0,0.0,0.0
1,Released,65.0,1.0,4.0,4.0,0.0,11.0,31.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Executing,4.0,0.0,2.0,0.0,1.0,1.0,10.0,0.0,0.0
1,Released,65.0,1.0,4.0,4.0,0.0,11.0,31.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()